<a href="https://colab.research.google.com/github/soumitrapy/da6401_assignment2/blob/main/partA/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !git clone https://github.com/soumitrapy/da6401_assignment2.git project
# %cd project
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q nature_12K.zip -d data/

Cloning into 'project'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 45 (delta 16), reused 36 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 25.15 KiB | 12.58 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/project/project
--2025-04-22 09:47:14--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.98.207, 142.251.107.207, 74.125.196.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G   141MB/s    in 37s     

2025-04-22 09:47:51 (98.1 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [4]:
# !git pull
%cd partA

/content/project/project/partA


In [5]:
import wandb
import yaml
import os
from datetime import datetime

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset,DataLoader, random_split

from torchvision import transforms
#from torchvision.io import read_image

In [7]:
import yaml
config = yaml.safe_load(open("config/smallcnn.yaml"))

config['dataset']['batch_size']=100
config['train']['epochs']=1
config['use_wandb']=True
config


{'project': 'dla2parta',
 'use_wandb': True,
 'dataset': {'name': 'CustomDataset',
  'path': '../data/inaturalist_12K/',
  'img_size': 256,
  'class_names': ['Plantae',
   'Mammalia',
   'Animalia',
   'Reptilia',
   'Amphibia',
   'Aves',
   'Fungi',
   'Arachnida',
   'Mollusca',
   'Insecta'],
  'batch_size': 100},
 'train': {'epochs': 1, 'val_interval': 1},
 'model': {'name': 'simplecnn',
  'in_channels': 3,
  'num_classes': 10,
  'num_layers': 5,
  'filters': [16, 32, 64, 32, 16],
  'kernel_size': 3,
  'activation': 'relu',
  'dense_neurons': 100}}

In [8]:
import wandb
if config.get('use_wandb',False):
    wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: soumitrapy (soumitrapy-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### DataLoader Creation

In [9]:
from preprocessing import CustomDataset
from torchvision import transforms
import torch

# Data augmentation and normalization for training
# Just normalization for validation
cfg = config['dataset']

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
splits = ['train', 'val']
datasets = {x:CustomDataset(path= cfg['path']+x,
                            class_names=cfg['class_names'],
                            transform=data_transforms[x])
            for x in splits}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x],
                                              batch_size=cfg['batch_size'],
                                              shuffle=True
                                              )
                for x in splits}

class_names = datasets['train'].class_names
print(f"train len: {len(datasets['train'])}, val len: {len(datasets['val'])}")

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"device = {device}")

train len: 9999, val len: 2000
device = cuda


In [10]:
for x, y in dataloaders['train']:
    print(x.shape,y.shape)
    break

torch.Size([100, 3, 256, 256]) torch.Size([100])


### Model

In [11]:
from models.smallcnn import SmallCNN

cfg = config['model']
model = SmallCNN(cfg)
model(x).shape

torch.Size([100, 10])

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)

### Training Loop

In [20]:
from train import train
if config['use_wandb']:
    run_name = type(model).__name__+str(datetime.now())[8:18]
    wandb.init(
        project=config['project'],
        name = run_name,
        config=config['model'],
    )
train(model=model, optimizer=optimizer, loss_fn=loss_fn, dataloaders=dataloaders,config=config['train'], model_config=config['model'], scheduler = scheduler, device = device, use_wandb = config['use_wandb'])

Validation: 100%|██████████| 20/20 [00:23<00:00,  1.17s/it, val_accuracy=24.4, val_loss=2.1]


In [18]:
wandb.init(
    project=config['project'],
    name = run_name,
    config=config['model'],
)

### Prediction

In [19]:
# model_artifact = wandb.use_artifact("trained-model:latest")
# model_dir = model_artifact.download()
# model_path = os.path.join(model_dir, "trained_model.pt")
# model_config = model_artifact.metadata

# model = SmallCNN(model_config)
# model.load_state_dict(torch.load(model_path))
import train

model.to(device)
loss, accuracy, highest_losses, hardest_examples, true_labels, preds = train.evaluate(model, dataloaders['val'])

wandb.summary.update({"loss": loss, "accuracy": accuracy})

wandb.log({"high-loss-examples":
    [wandb.Image(hard_example, caption=str(int(pred)) + "," +  str(int(label)))
        for hard_example, pred, label in zip(hardest_examples, preds, true_labels)]})


CommError: artifact 'trained-model:latest' not found in 'soumitrapy-iit-madras/dla2parta'

In [21]:
from importlib import reload
reload(train)

ImportError: module train not in sys.modules